# Linear Regression

In [2]:
%matplotlib inline
import math
import time
import numpy as np
import torch
from d2l import torch as d2l

## Basics

Regression is whenever we want to predict a value based on some number of features. In the following example, assume we wish to predict the value (in dollars) of houses based on their size (in sq. feet) and age (in years). First we have to get our hands on some training data. Each row of this training set is an example. The house prices in this training set are the labels/targets. The age and price are the features/covariates. 

Linear regression is one of the simplest amnd most popular standard tools. Assume the relationsip between the features $\mathbf{x}$ and $y$ is approximately linear, but allow for some random noise, which we assume to be Gaussian. 

Use $n$ to represent the number of _examples_ in our training dataset. Use superscripts to enumerate samples and targets, and subscripts to index coordinates. $\mathbf{x}^{(i)}$ indicates the vector of features for the $i$th sample/example. $x^{(i)}_j$ indicates its $j$th coordinate/feature. 

### Model

The assumption of linearity means that the expectation value of the target may be expressed in terms of a weighted sum of the features, that is:

$$ \hat{y}^{(i)} = w^{(i)}_{age}x^{(i)}_{age} + w^{(i)}_{sqft}x^{(i)}_{sqft} + b$$

Where w are the weights and b is the bias, or offset, representing the intercept of the linear model with the y axis. Although there is no "real" house with a square footage of 0, this value is still relevant for ensuring that the model is fit correctly. Presumably our assumption of linearity would break down outside of the limits of a normal/acceptable house size.